In [1]:
import geopandas as gpd
import pandas as pd
import rtree

In [63]:
ac_parcels_poly = gpd.read_file("../../data/raw/ac_parcels")
ac_parcels_poly.crs = 'epsg:4326' # set the CRS of the shapefile
#ac_parcels_poly = ac_parcels_poly.to_crs(epsg=3857) # transform to a projected geometry for operations
ac_parcels_poly_sindex = ac_parcels_poly.sindex

In [64]:
d = {'lat': [37.7501568], 'long': [-122.1941158],'location': ['AB&I']} # sample df with coordinates for AB&I Foundry in East Oakland
df = pd.DataFrame(data=d)
pt_data = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(
        x=df["long"], y=df["lat"]))
pt_data.crs = 'epsg:4326' # set the CRS of the shapefile
#pt_data = pt_data.to_crs(epsg=3857) # transform to a projected geometry for operations

In [71]:
def find_closest_parcel(row):
    point = row["geometry"]
    bounds = point.bounds
    possible_matches_idx = list(ac_parcels_poly_sindex.intersection(point.bounds))
    possible_matches = ac_parcels_poly.iloc[possible_matches_idx]
    hits = possible_matches[possible_matches.intersects(point)]
    
    closest_idx = None
    closest_dist = float("inf")    
    if len(hits) == 1:   # a single overlapping parcel
        print("Single overlapping")
        closest_idx = int(hits.index[0])
        closest_dist = 0
    elif len(hits) > 1:  # multiple overlapping parcels: pick one with closest centroid
        print("Multiple overlapping")
        dist_to_centroids = [point.distance(centroid) for centroid in hits.centroid]
        closest = np.argmin(dist_to_centroids)
        closest_idx = int(hits.index[closest])
        closest_dist = 0
    else:                # no matches, grab nearest parcel in r-tree
        print("No matches, grabbing closest")
        closest_idx = list(ac_parcels_poly_sindex.nearest(point.bounds, 1))[0]
        closest_dist = ac_parcels_poly.loc[int(closest_idx)]["geometry"].distance(point)
    return (closest_idx, closest_dist)

pt_data[["parcel_idx", "parcel_dist"]] = pt_data.apply(
    lambda x: find_closest_parcel(x), axis=1, result_type="expand")

No matches, grabbing closest


In [70]:
ac_parcels_poly_sindex

rtree.index.Index(bounds=[-122.30457081355219, 37.57487269957006, -121.745084219451, 37.88489315224816], size=1000)

In [68]:
ac_parcels_poly.iloc[354]

fid_parcel                                                  348
book                                                         45
page                                                       5389
clca_categ                                                 None
date_updat                                           2004-08-09
centroid_y                                      2092305.2523999
sub_parcel                                                 None
apn_sort                                          045 538902500
parcel                                                       25
centroid_x                                     6076194.17943388
date_creat                                           2004-08-09
apn                                                  45-5389-25
comments                                                   None
geometry      POLYGON ((-122.1786108903112 37.72916360817982...
Name: 354, dtype: object